In [ ]:
import os
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.stattools import adfuller
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
seed = 1

# 경로지정
os.chdir(r'D:\QAE\fall\\머신러닝')

# df
df = pd.read_excel(r'기초데이터.xlsx')

# 데이터 null값 있나 확인 후 있으면 ffil(전일 데이터로 채우기)
null_check = df.isnull().values.any()
print(null_check) # True = > null 값 존재
df.fillna(method='ffill', inplace=True)

# 재확인
null_check = df.isnull().values.any()
print(null_check) # False = > null 값 없음

# 환율 변동 1개월, 3개월, 6개월, 1년 => vix지수와 관련?
# USD_VND : 베트남동, USD_SGD : 싱가폴 달러, USD_CNY : 중국 위안, USD_IDR: 인도네시아루피아
# YEN_DOLLAR : 일본 엔, WON_DOLLAR : 대한민국 원, EURO_USD: 유로

# 일별 변화율 계산
df['USD_VND_daily'] = (df['USD_VND'].shift(0) - df['USD_VND'].shift(1)) / df['USD_VND'].shift(1)
df['USD_SGD_daily'] = (df['USD_SGD'].shift(0) - df['USD_SGD'].shift(1)) / df['USD_SGD'].shift(1)
df['USD_CNY_daily'] = (df['USD_CNY'].shift(0) - df['USD_CNY'].shift(1)) / df['USD_CNY'].shift(1)
df['USD_IDR_daily'] = (df['USD_IDR'].shift(0) - df['USD_IDR'].shift(1)) / df['USD_IDR'].shift(1)
df['YEN_DOLLAR_daily'] = (df['YEN_DOLLAR'].shift(0) - df['YEN_DOLLAR'].shift(1)) / df['YEN_DOLLAR'].shift(1)
df['WON_DOLLAR_daily'] = (df['WON_DOLLAR'].shift(0) - df['WON_DOLLAR'].shift(1)) / df['WON_DOLLAR'].shift(1)
df['EURO_USD_daily'] = (df['EURO_USD'].shift(0) - df['EURO_USD'].shift(1)) / df['EURO_USD'].shift(1)

"""
# 각 환율의 1개월, 3개월, 6개월, 1년 변동성 계산하기
ticker = ['USD_VND', 'USD_SGD', 'USD_CNY', 'USD_IDR', 'YEN_DOLLAR', 'WON_DOLLAR', 'EURO_USD']

df = df.dropna().reset_index(drop='index')

for money in ticker:

    print(f'{money}에 대한 계산 처리중...')

    # 월
    df[f'{money}_monthly'] = df[f'{money}_daily'].rolling(30).std()

    # 분기
    df[f'{money}_quarterly'] = df[f'{money}_daily'].rolling(60).std()

    # 반기
    df[f'{money}_halfly'] = df[f'{money}_daily'].rolling(180).std()

    # 연
    df[f'{money}_yearly'] = df[f'{money}_daily'].rolling(360).std()


# 데이터 너무 많음 월, 분기, 반기, 연으로 나누겠음
df_month = df[['날짜', 'VIX', 'USD_VND_monthly', 'USD_SGD_monthly', 'USD_CNY_monthly', 'USD_IDR_monthly', 'YEN_DOLLAR_monthly', 'WON_DOLLAR_monthly', 'EURO_USD_monthly']]
df_quarter = df[['날짜', 'VIX', 'USD_VND_quarterly', 'USD_SGD_quarterly', 'USD_CNY_quarterly', 'USD_IDR_quarterly', 'YEN_DOLLAR_quarterly', 'WON_DOLLAR_quarterly', 'EURO_USD_quarterly']]
df_half = df[['날짜', 'VIX', 'USD_VND_halfly', 'USD_SGD_halfly', 'USD_CNY_halfly', 'USD_IDR_halfly', 'YEN_DOLLAR_halfly', 'WON_DOLLAR_halfly', 'EURO_USD_halfly']]
df_year = df[['날짜', 'VIX', 'USD_VND_yearly', 'USD_SGD_yearly', 'USD_CNY_yearly', 'USD_IDR_yearly', 'YEN_DOLLAR_yearly', 'WON_DOLLAR_yearly', 'EURO_USD_yearly']]

# 월부터
df_month = df_month.dropna().reset_index(drop='index')
df_month.index = df_month['날짜']
df_month.drop(columns='날짜', inplace=True)
"""
# 각 환율의 변동성을 그림으로 나타내기 => 걍 변동성 일정하게 움직이는거 확인하려고 그리는듯, 자료로 쓰려고 만들어놈
# 베트남 동
plt.figure(figsize=(18,18))
plt.plot(df['날짜'], df['USD_VND_daily'])
plt.show()

#싱가폴달러
plt.figure(figsize=(18,18))
plt.plot(df['날짜'], df['USD_SGD_daily'])
plt.show()

#차이나위안
plt.figure(figsize=(18,18))
plt.plot(df['날짜'], df['USD_CNY_daily'])
plt.show()

#인도네시아 루피아
plt.figure(figsize=(18,18))
plt.plot(df['날짜'], df['USD_IDR_daily'])
plt.show()

#엔달러
plt.figure(figsize=(18,18))
plt.plot(df['날짜'], df['YEN_DOLLAR_daily'])
plt.show()

#원달러
plt.figure(figsize=(18,18))
plt.plot(df['날짜'], df['WON_DOLLAR_daily'])
plt.show()

#유로달러
plt.figure(figsize=(18,18))
plt.plot(df['날짜'], df['EURO_USD_daily'])
plt.show()

# autocorrelation 확인, 왜? => 표본의 무작위성(randomness)은 서로 다른 관측치들에 대한 오차항이 상관되어 있지 말아야함을 의미하나, 자기상관은 이런 무작위성이 위배됨.
# 엄청나게 significant한 상관성을 보이지는 않음..., 일부는 상관성 존재 => garch로 변동성 분포 확인
# 베트남 동
plot_acf(df[['USD_VND_daily']].iloc[-365:].values) # 주기모호

# 싱가폴 달러
plot_acf(df[['USD_SGD_daily']].iloc[-365:].values) # 주기모호

# 차이나위안
plot_acf(df[['USD_CNY_daily']].iloc[-365:].values) # 주기가 모호함

# 인도네시아 루피아
plot_acf(df[['USD_IDR_daily']].iloc[-365:].values) # 주기가 모호함

# 엔달러
plot_acf(df[['YEN_DOLLAR_daily']].iloc[-365:].values) # 주기가 모호함

# 원달러
plot_acf(df[['WON_DOLLAR_daily']].iloc[-365:].values) # 주기가 모호함

#유로달러
plot_acf(df[['EURO_USD_daily']].iloc[-365:].values) # 주기 모호함

# data의 stationary test를 위해 ADF 검정  실시해보겠음
# H0: non-stationary, H1: stationary
def adf_test(df):

    result = adfuller(df.values)

    print(f'ADF Statistics: {result[0]}')
    print(f'p-value: {result[1]}')
    print(f'Critical values:')
    for key, value in result[4].items():
        print('\t%s: .%.3f' % (key, value))

# 베트남 동
adf_test(df['USD_VND_daily'].iloc[1:]) # stationary

# 싱가폴 달러
adf_test(df['USD_SGD_daily'].iloc[1:]) # stationary

# 차이나위안
adf_test(df['USD_CNY_daily'].iloc[1:]) # stationary

# 인도네시아 루피아
adf_test(df['USD_IDR_daily'].iloc[1:]) # stationary

# 엔달러
adf_test(df['YEN_DOLLAR_daily'].iloc[1:]) # stationary

# 원달러
adf_test(df['WON_DOLLAR_daily'].iloc[1:]) # stationary

#유로달러
adf_test(df['EURO_USD_daily'].iloc[1:])  # stationary

# stationary하고 자기상관성 없는데 굳이 garch 쓸 필요는 없어보임.
# 따라서 모델 해석에 도움이 되기 위해서 Linear model을 사용해서 예측을 진행
# 첫번쨰 training data와 test데이터의 train 80%, test 20%로 나눈후 한 번에 예측을 결정하는 일반적인 방법 => 파라미터 간의 관계 해석을 위해
# 해석적인 영역에서는 분명 도움이 될 것이라고 생각됨.
# 두번째 방식은 lookback 즉 학습기간을 월, 분기, 반기, 연으로 지정해서 그 다음 일을 예측하는 방식을 사용해보겠음

ticker = ['USD_VND_daily', 'USD_SGD_daily', 'USD_CNY_daily', 'USD_IDR_daily', 'YEN_DOLLAR_daily', 'WON_DOLLAR_daily', 'EURO_USD_daily']

for name in ticker:

    print(f'{name}을 처리중')

    df[f'{name}'] = df[f'{name}'].shift(1)

# 첫번째 정석적인 머신러닝 방식을 통한 방법 => cv fold 방식은 시계열 자료이기 때문에 사용하지 않겠음.. => 변수간 관계 파악을 위해 용이하다고 판단됨..
df = df.dropna().reset_index(drop='index')
df.index = df['날짜']
df.drop(columns=['날짜'], inplace=True)

# 어차피 변화율들이기 때문에 log를 사용할 필요는 없어보임..
X_data = df[['USD_VND_daily', 'USD_SGD_daily', 'USD_CNY_daily', 'USD_IDR_daily', 'YEN_DOLLAR_daily', 'WON_DOLLAR_daily', 'EURO_USD_daily']].values
y_data = df[['VIX']].values

X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, shuffle=False, random_state=seed)

# train
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

# prediction
vix_pred = linear_model.predict(X_test)

# coefficients, ticker 순서대로 coef임..
print(ticker)
print("Coefficients: \n", linear_model.coef_)

# Mean Squared Error
print("Mean squared error: %.2f" % mean_squared_error(y_test, vix_pred))

# r2_score
print("Coefficient of determination: %.2f" % r2_score(y_test, vix_pred))

plt.plot(y_test)
plt.plot(vix_pred)
# 제대로된 예측을 할 수 없음이 보임..


# 그래서 lookback기간 즉 데이터 학습 기간을 30일로 놓고 그다음일을 예측하는 방식으로 누적 예측 사용
lookback = 30 # <- 각각 30, 90, 180, 360을 지정해서 테스트
performance_list = []
for i in range(len(df)):

    print(f'{i}' + '처리중...')

    # lookback기간보다 작거나 같으면pass
    if i < lookback+1:
        pass

    else:

        tmp = df.iloc[i-lookback:i+1]

        X_train = tmp[['USD_VND_daily', 'USD_SGD_daily', 'USD_CNY_daily', 'USD_IDR_daily', 'YEN_DOLLAR_daily', 'WON_DOLLAR_daily', 'EURO_USD_daily']].iloc[:lookback]
        y_train = tmp[['VIX']].iloc[:lookback]



        X_test = tmp[['USD_VND_daily', 'USD_SGD_daily', 'USD_CNY_daily', 'USD_IDR_daily', 'YEN_DOLLAR_daily', 'WON_DOLLAR_daily', 'EURO_USD_daily']].iloc[-1].values.reshape(1,-1)
        y_test = tmp[['VIX']].iloc[-1].values.reshape(1,-1)

        # train
        lin_reg = LinearRegression()
        lin_reg.fit(X_train, y_train)

        # test
        prediction = lin_reg.predict(X_test)

        performance = {'날짜': tmp.index[-1], '실제VIX': y_test[0][0], '예측vix': prediction[0][0], '계수': lin_reg.coef_}

        # append
        performance_list.append(performance)

# 데이터프레임화
performance_frame = pd.DataFrame(performance_list)


performance_frame['예측vix'] = performance_frame['예측vix'].astype(float)
performance_frame['실제VIX'] = performance_frame['실제VIX'].astype(float)


# Mean Squared Error
print("Mean squared error: %.2f" % mean_squared_error(performance_frame['실제VIX'], performance_frame['예측vix']))

# r2_score
print("Coefficient of determination: %.2f" % r2_score(performance_frame['실제VIX'], performance_frame['예측vix']))

# error 높고, R2 낮음 그렇지만 어느정도 팔로윙하는모습 => 변동성 낮을때는 덜 민감하다가, 변동성이 높은 구간엔 훨씬 많이 민감해짐
def performance_graph(df):

    fig = plt.figure(figsize=(14, 10))

    plt.suptitle(
        f"MSE={mean_squared_error(df['실제VIX'], df['예측vix'])}, R2={r2_score(df['실제VIX'], df['예측vix'])}")

    ax = fig.add_subplot(1, 1, 1)
    ax.plot(df.index, df['실제VIX'], label='Real_VIX')
    ax.plot(df.index, df['예측vix'], label='Prediction_vix')
    plt.legend()

performance_graph(df=performance_frame)

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
